In [70]:
import numpy as np

def top_k_indices(matrix, k, axis=1):
    """
    返回二维数组中每行或每列前k个最大元素的索引。
    
    参数:
        matrix (np.ndarray): 需要检索的二维NumPy数组
        k (int): 需要返回的最大值的数量
        axis (int): 指定操作的轴，axis=1表示处理每行，axis=0表示处理每列
    
    返回:
        np.ndarray: 包含每行或每列前k个最大元素索引的二维数组
    """
    # 检查k是否超出轴的长度，如果超出则使用该轴的长度
    if axis == 1:  # 处理每行
        k = min(k, matrix.shape[1])
        indices = np.argsort(matrix, axis=1)[:, -k:][:, ::-1]
    elif axis == 0:  # 处理每列
        k = min(k, matrix.shape[0])
        indices = np.argsort(matrix, axis=0)[-k:, :][::-1, :]
    return indices

# 示例数据和使用函数
probs = np.random.rand(10, 20)  # 创建一个10x20的二维数组
k = 5
top_indices = top_k_indices(probs, k)
print("Indices of top", k, "elements in each row:\n", top_indices)
print("Top", k, "elements in each row:\n", np.take_along_axis(probs, top_indices, axis=1))


Indices of top 5 elements in each row:
 [[ 6 11  1 10  3]
 [ 7  5 12 15  1]
 [15  5  9  6 19]
 [ 1 15 19  6 11]
 [ 2 16 19  1 17]
 [ 3  4  0 10 14]
 [13 15 12 18  7]
 [12 16 19  7  1]
 [19  6  9  7  8]
 [18 19  0  1  9]]
Top 5 elements in each row:
 [[0.99581529 0.96492494 0.9485403  0.94385086 0.84834527]
 [0.96920587 0.85324598 0.82371767 0.79122743 0.72754361]
 [0.97509468 0.96551664 0.9416678  0.85900264 0.77450095]
 [0.9673378  0.96705492 0.92929167 0.91328434 0.88502185]
 [0.98510868 0.90623142 0.85180858 0.82814661 0.81636431]
 [0.997994   0.98788917 0.96239507 0.95612316 0.94782189]
 [0.95556839 0.82680527 0.8002556  0.7510323  0.70641058]
 [0.98385383 0.94462572 0.92539503 0.92345554 0.89270716]
 [0.94218472 0.92797617 0.91145088 0.86686091 0.81615075]
 [0.98299913 0.98224778 0.88826504 0.86031068 0.81642103]]


In [84]:
import numpy as np

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)

def recall_at_k(r, k, all_pos_num):
    r = np.asfarray(r)[:k]
    if all_pos_num == 0:
        return 0
    else:
        return np.sum(r) / all_pos_num

def dcg_at_k(r, k, method=1):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k, method=1):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

def hit_at_k(r, k):
    r = np.array(r)[:k]
    if np.sum(r) > 0:
        return 1.
    else:
        return 0.

def get_performance(user_pos_test, r, Ks):
    precision, recall, ndcg, hit_ratio = [], [], [], []

    for K in Ks:
        precision.append(precision_at_k(r, K))
        recall.append(recall_at_k(r, K, len(user_pos_test)))
        ndcg.append(ndcg_at_k(r, K))
        hit_ratio.append(hit_at_k(r, K))

    return {'recall': np.array(recall), 'precision': np.array(precision),
            'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio)}

# 转换索引为二元相关性数组
def get_binary_relevance(user_interests, num_items):
    relevance = np.zeros(50000)
    relevance[user_interests] = 1
    return relevance

user_pos_test = np.array([2, 5, 78])
predicted_order = np.array([5, 2, 1, 0, 9, 8, 3, 4, 78])
# r = get_binary_relevance(user_pos_test, len(predicted_order))
Ks = [5, 10, 20, 50]
results = get_performance(user_pos_test, r, Ks)
results

{'recall': array([0.33333333, 0.66666667, 0.66666667, 0.66666667]),
 'precision': array([0.2 , 0.2 , 0.1 , 0.04]),
 'ndcg': array([0.23463936, 0.40179982, 0.40179982, 0.40179982]),
 'hit_ratio': array([1., 1., 1., 1.])}

In [85]:
user_pos_test, predicted_order

(array([ 2,  5, 78]), array([ 5,  2,  1,  0,  9,  8,  3,  4, 78]))

In [33]:
import numpy as np

def get_binary_relevance(rank_indices, ground_truth_indices, data_size):
    """
    给定排名索引和实际索引，返回一个二进制相关性数组。
    """
    relevance = np.zeros(data_size, dtype=int)
    relevance[rank_indices] = 1
    ground_truth_relevance = np.zeros(data_size, dtype=int)
    ground_truth_relevance[ground_truth_indices] = 1
    return relevance, ground_truth_relevance

def calculate_ndcg(relevance, ground_truth_relevance, k):
    """
    计算NDCG@k。
    """
    def dcg(relevance_scores):
        """辅助函数，用于计算DCG。"""
        return np.sum((2 ** relevance_scores - 1) / np.log2(np.arange(2, len(relevance_scores) + 2)))
    
    dcg_val = dcg(relevance[:k])
    idcg_val = dcg(ground_truth_relevance[:k])
    return dcg_val / idcg_val if idcg_val > 0 else 0

def calculate_recall(predicted_rank_indices, ground_truth_indices, k):
    """
    Calculate Recall@k.
    
    Parameters:
    - predicted_rank_indices: The indices of items as they are ranked by the prediction.
    - ground_truth_indices: The indices of items that are actually relevant.
    - k: The number of top-ranked items to consider for calculation of Recall.
    
    Returns:
    - Recall@k as a float.
    """
    # Create a set of the top K predicted indices
    top_k_predicted_set = set(predicted_rank_indices[:k])
    
    # Create a set of all the ground truth relevant indices
    ground_truth_set = set(ground_truth_indices)
    
    # Calculate the number of hits in the top K predictions
    hits_in_top_k = len(top_k_predicted_set.intersection(ground_truth_set))
    
    # Calculate Recall@K
    return hits_in_top_k / len(ground_truth_set) if ground_truth_set else 0

# 假设数据
predicted_rank_indices = np.array([1, 23, 3, 4, 5])
ground_truth_indices = np.array([100, 10, 1, 8, 4])
data_size = max(np.max(predicted_rank_indices), np.max(ground_truth_indices)) + 1  # 根据最大索引确定数组大小

# 计算二进制相关性分数
relevance, ground_truth_relevance = get_binary_relevance(predicted_rank_indices, ground_truth_indices, data_size)

# 计算NDCG和Recall
k = 5  # 计算前5个的指标
ndcg_value = calculate_ndcg(relevance, ground_truth_relevance, k)
recall_value = calculate_recall(relevance, ground_truth_relevance, k)

print(f"NDCG@{k}: {ndcg_value:.4f}")
print(f"Recall@{k}: {recall_value:.4f}")


NDCG@5: 1.4232
Recall@5: 1.0000


In [4]:
import pickle
import pandas as pd
with open('../BigDataSource/Teddy2024/附件1/image_features_cpu.pkl', 'rb') as f:
    all_image_features = pickle.load(f)

In [6]:
with open('../BigDataSource/Teddy2024/附件1/text_features_cpu.pkl', 'rb') as f:
    all_text_features = pickle.load(f)

In [15]:
data = pd.read_csv('../BigDataSource/Teddy2024/附件1/ImageWordData.csv').to_numpy()

In [16]:
all_image_features[0][0]

'Image14001004-7525.jpg'

In [19]:
all_text_features[0][0]

'只看阵容对手就已经吓尿了,巴西队黄金一代神挡杀神,佛挡杀佛!'

In [24]:
import numpy as np

# 已知的列索引
col_index = 0

# 目标值
target_value = all_image_features[0][0]

# 使用np.where找到特定列中特定值的行索引
row_indices = np.where(data[:, col_index] == target_value)[0]

# 输出行索引
print("Row indices where column {} is {}: {}".format(col_index, target_value, row_indices))


Row indices where column 0 is Image14001004-7525.jpg: [29482]


In [25]:
import numpy as np

# 已知的列索引
col_index = 1

# 目标值
target_value = all_text_features[0][0]

# 使用np.where找到特定列中特定值的行索引
row_indices = np.where(data[:, col_index] == target_value)[0]

# 输出行索引
print("Row indices where column {} is {}: {}".format(col_index, target_value, row_indices))


Row indices where column 1 is 只看阵容对手就已经吓尿了,巴西队黄金一代神挡杀神,佛挡杀佛!: [1]


In [47]:
all_image_idx = []
all_text_idx = []
for i in range(len(all_text_features)):
    all_image_idx.append(all_image_features[i][0])
    all_text_idx.append(all_text_features[i][0])

In [48]:
from tqdm import * 

new_image_features = []
new_text_features = []
for i in trange(len(data)):
    image_idx = all_image_idx.index(data[i][0])
    text_idx = all_text_idx.index(data[i][1])
    new_image_features.append([data[i][0], data[i][1], all_image_features[image_idx][1]])
    new_text_features.append([data[i][0], data[i][1], all_text_features[text_idx][1]])
    # print(image_idx, text_idx)

100%|██████████| 50000/50000 [00:32<00:00, 1544.01it/s]


In [88]:
import numpy as np

def recall_at_k(relevant, recommended, k):
    """
    计算 Recall@K
    :param relevant: 实际感兴趣的项目矩阵，每行代表一个用户的相关项目集合，二维数组
    :param recommended: 推荐系统给出的推荐项目矩阵，每行代表一个用户的推荐项目列表，二维数组
    :param k: 考虑的推荐列表的前K项
    :return: 每个用户的 Recall@K 的值，一维数组
    """
    recalls = []
    for rel_set, rec_list in zip(relevant, recommended):
        if k < len(rec_list):
            rec_list_k = rec_list[:k]
        else:
            rec_list_k = rec_list
        relevant_recommended = set(rel_set).intersection(rec_list_k)
        if len(rel_set) > 0:
            recalls.append(len(relevant_recommended) / len(set(rel_set)))
        else:
            recalls.append(0)
    return np.array(recalls)

def dcg_at_k(scores, k):
    """
    计算 DCG@K
    :param scores: 相关性得分列表，列表类型
    :param k: 考虑的推荐列表的前K项
    :return: DCG@K 的值
    """
    scores = scores[:k]
    if len(scores):
        return np.sum(scores / np.log2(np.arange(2, len(scores) + 2)))  # log base 2 of positions 2 through k+1
    return 0

def ndcg_at_k(relevant, recommended, k):
    """
    计算 NDCG@K
    :param relevant: 实际感兴趣的项目矩阵，二维数组
    :param recommended: 推荐项目矩阵，二维数组
    :param k: 考虑的推荐列表的前K项
    :return: 每个用户的 NDCG@K 的值，一维数组
    """
    ndcgs = []
    for rel_set, rec_list in zip(relevant, recommended):
        rel_set = set(rel_set)
        if k < len(rec_list):
            rec_list_k = rec_list[:k]
        else:
            rec_list_k = rec_list
        
        actual_scores = [1 if item in rel_set else 0 for item in rec_list_k]
        ideal_scores = [1] * min(len(rel_set), k)
        
        actual_dcg = dcg_at_k(actual_scores, k)
        ideal_dcg = dcg_at_k(ideal_scores, k)
        
        if ideal_dcg > 0:
            ndcgs.append(actual_dcg / ideal_dcg)
        else:
            ndcgs.append(0)
    return np.array(ndcgs)

# 示例数据
n = 10  # 假设有5个用户
np.random.seed(0)
relevant = np.random.randint(1, 500, (n, 100))
recommended = np.random.randint(1, 500, (n, 100))

# 计算 Recall@K 和 NDCG@K
k_value = 8
recalls = recall_at_k(relevant, recommended, k_value)
ndcgs = ndcg_at_k(relevant, recommended, k_value)

print("Recalls@3:", recalls)
print("NDCGs@3:", ndcgs)


Recalls@3: [0.03488372 0.04395604 0.0212766  0.01098901 0.03296703 0.02083333
 0.01075269 0.02173913 0.         0.01041667]
NDCGs@3: [0.50263179 0.40530239 0.21078559 0.09010001 0.33723521 0.16410877
 0.07979454 0.26852557 0.         0.12647135]


In [87]:
relevant.shape

(10, 100)

In [67]:
recommended

array([[243, 359,  21, 172, 119, 129, 485, 366, 106, 195, 226, 141, 272,
        119,  34, 152, 397, 390,  39, 258, 267,   7, 382, 263, 103, 423,
        332, 371, 342, 383, 371, 179, 308, 259, 333, 158,  10, 116, 457,
        390, 117, 182, 236, 194, 300,  80,  63, 333,  48,  77, 150, 121,
        275, 346,  14, 223,  93, 237, 106, 214,   7, 101,  49,  41, 411,
        159, 390, 144, 448, 486,  38,  60, 114,  96, 353, 497, 209, 289,
         63, 395, 397,  94, 321, 414, 347, 488, 151, 404, 390, 293,  84,
        128, 485, 262, 349, 461, 155, 242,  84, 397],
       [487, 136, 181,  81, 414,   1, 216, 208, 331, 253, 132,  62, 324,
        145, 369, 458,  85, 117, 230, 256, 313, 319, 441,  49, 146, 374,
        442,  10, 493, 191,  80, 231, 207, 418, 389, 401, 229, 328, 319,
        280, 348, 367,   8, 279, 355, 159, 247,  81, 156,  48, 496, 115,
         12, 472, 335, 488, 287,  91, 291,  55, 338, 164,  72, 366, 316,
        113, 422,  69, 438,  14, 102, 276, 364, 265, 353, 210, 198, 38

In [68]:
recalls

array([0.03488372, 0.04395604, 0.0212766 , 0.01098901, 0.03296703,
       0.02083333, 0.01075269, 0.02173913, 0.        , 0.01041667])

In [2]:
from cn_clip.clip import load_from_name

model, preprocess = load_from_name("ViT-B-16", download_root='../BigDataSource/', vision_model_name='ViT-B-16',text_model_name='RoBERTa-wwm-ext-base-chinese')

Loading vision model config from /Users/zengyuxiang/Documents/科研代码/CLIP_finetune/cn_clip/clip/model_configs/ViT-B-16.json
Loading text model config from /Users/zengyuxiang/Documents/科研代码/CLIP_finetune/cn_clip/clip/model_configs/RoBERTa-wwm-ext-base-chinese.json
Model info {'embed_dim': 512, 'image_resolution': 224, 'vision_layers': 12, 'vision_width': 768, 'vision_patch_size': 16, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 12, 'text_type_vocab_size': 2}


In [ ]:
vision